In [1]:
import pandas as pd
import numpy as np 
import dabl
import seaborn as sns
import matplotlib.pyplot as plt

## join fte+scores to cencus_poverty 

Notebook # 3 

- 1 = connect scores to school labels
- 2 = connect labeled scores to staffing-- and district data? (fte)
- 3 = connect all school data to community poverty data from cencus
- 4= connect all school community data to yelp community data

In [11]:
comdf = pd.read_csv('../../data/processed/all_school_data.csv')
comdf.drop(columns=['Unnamed: 0','Type ID_y','Test Year_y','_merge'],inplace=True)
comdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29239 entries, 0 to 29238
Data columns (total 56 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   County Code                                                         29239 non-null  int64  
 1   District Code                                                       29239 non-null  int64  
 2   School Code                                                         29239 non-null  int64  
 3   Test Year_x                                                         29239 non-null  int64  
 4   Student Group ID                                                    29239 non-null  int64  
 5   Total Tested at Reporting Level                                     28019 non-null  float64
 6   Total Tested with Scores at Reporting Level                         28002 non-null  float64
 7   Grade        

In [3]:
## check nulls 
m = pd.DataFrame([comdf.isnull().sum(),100*comdf.isnull().mean()])
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head(15)


,count,%
Mean Scale Score,12395.0,42.392011
Area 4 Percentage Near Standard,10268.0,35.117480
Area 2 Percentage Below Standard,10268.0,35.117480
Area 1 Percentage Below Standard,10268.0,35.117480
Area 1 Percentage Above Standard,10268.0,35.117480
Area 2 Percentage Near Standard,10268.0,35.117480
Area 2 Percentage Above Standard,10268.0,35.117480
Area 3 Percentage Above Standard,10268.0,35.117480
Area 3 Percentage Near Standard,10268.0,35.117480
Area 3 Percentage Below Standard,10268.0,35.117480


In [4]:
cen = pd.read_excel('../../data/raw/ussd20.xls',header=2)
cen = cen[cen['State Postal Code']=='CA']
cen.rename(columns={'Name':'District Name'},inplace=True)
cen.rename(columns={'Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder':'est_num_u17_pov'},inplace=True)
cen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 942 entries, 644 to 1585
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   State Postal Code           942 non-null    object
 1   State FIPS Code             942 non-null    int64 
 2   District ID                 942 non-null    int64 
 3   District Name               942 non-null    object
 4   Estimated Total Population  942 non-null    int64 
 5   Estimated Population 5-17   942 non-null    int64 
 6   est_num_u17_pov             942 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 58.9+ KB


In [5]:
cen.sample(5)

,State Postal Code,State FIPS Code,District ID,District Name,Estimated Total Population,Estimated Population 5-17,est_num_u17_pov
1046,CA,6,20190,La Habra City Elementary School District,53820,6095,688
928,CA,6,14730,Fullerton Elementary School District,123499,12701,1298
1305,CA,6,31290,Pollock Pines Elementary School District,9772,863,96
851,CA,6,65,Dinuba Unified School District,30671,6662,1401
847,CA,6,11040,Denair Unified School District,8513,1731,187


In [6]:
## check nulls 
m = pd.DataFrame([cen.isnull().sum(),100*cen.isnull().mean()])
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head(15)


,count,%
State Postal Code,0.0,0.0
State FIPS Code,0.0,0.0
District ID,0.0,0.0
District Name,0.0,0.0
Estimated Total Population,0.0,0.0
Estimated Population 5-17,0.0,0.0
est_num_u17_pov,0.0,0.0


In [7]:
## clean up cencus column to match school data on district name

In [8]:
cen['District Name'] = cen['District Name'].astype(str).str.lower()
cen['District Name'] = cen['District Name'].str.replace('school district','')
cen['District Name'] = cen['District Name'].str.rstrip()
cen['District Name'].sample(4)

1257                pajaro valley joint unified
763                         cambrian elementary
994     hughes-elizabeth lakes union elementary
651                      alexander valley union
Name: District Name, dtype: object

## Join all school data to poverty data by district on District name

In [12]:
join = comdf.merge(cen,on='District Name',how='left',indicator=True)
join.drop(['State FIPS Code','State Postal Code'],axis=1,inplace=True)
join.rename(columns={'District ID':'cencus_district_ID'},inplace=True)
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30725 entries, 0 to 30724
Data columns (total 61 columns):
 #   Column                                                              Non-Null Count  Dtype   
---  ------                                                              --------------  -----   
 0   County Code                                                         30725 non-null  int64   
 1   District Code                                                       30725 non-null  int64   
 2   School Code                                                         30725 non-null  int64   
 3   Test Year_x                                                         30725 non-null  int64   
 4   Student Group ID                                                    30725 non-null  int64   
 5   Total Tested at Reporting Level                                     29489 non-null  float64 
 6   Total Tested with Scores at Reporting Level                         29472 non-null  float64 
 7   Grad

In [13]:
join['_merge'].value_counts()

both          22541
left_only      8184
right_only        0
Name: _merge, dtype: int64

In [14]:
m = pd.DataFrame([join.isnull().sum(),100*join.isnull().mean()])
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head(15)

,count,%
Mean Scale Score,12829.0,41.754272
Area 3 Percentage Above Standard,10620.0,34.564687
Area 1 Percentage Near Standard,10620.0,34.564687
Area 4 Percentage Below Standard,10620.0,34.564687
Area 4 Percentage Near Standard,10620.0,34.564687
Area 4 Percentage Above Standard,10620.0,34.564687
Area 3 Percentage Below Standard,10620.0,34.564687
Area 3 Percentage Near Standard,10620.0,34.564687
Area 2 Percentage Below Standard,10620.0,34.564687
Area 2 Percentage Near Standard,10620.0,34.564687


In [15]:
join[join['District Name']=='abc unified']

,County Code,District Code,School Code,Test Year_x,Student Group ID,Total Tested at Reporting Level,Total Tested with Scores at Reporting Level,Grade,Test ID,Students Enrolled,...,Total Number Operational Schools [Public School] 2020-21,State Name [District] 2020-21,"Total Students, All Grades (Excludes AE) [District] 2020-21",Full-Time Equivalent (FTE) Teachers [District] 2020-21,Pupil/Teacher Ratio [District] 2020-21,cencus_district_ID,Estimated Total Population,Estimated Population 5-17,est_num_u17_pov,_merge
6778,19,64212,6010862,2021,1,7.0,7.0,6,English Language Arts/Literacy,59.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6779,19,64212,6010862,2021,1,7.0,7.0,13,English Language Arts/Literacy,185.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6780,19,64212,6010862,2021,1,7.0,7.0,6,Mathematics,59.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6781,19,64212,6010862,2021,1,7.0,7.0,13,Mathematics,185.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6782,19,64212,6010904,2021,1,16.0,16.0,6,English Language Arts/Literacy,76.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6873,19,64212,6093447,2021,1,35.0,35.0,13,Mathematics,342.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6874,19,64212,6093454,2021,1,32.0,32.0,6,English Language Arts/Literacy,110.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6875,19,64212,6093454,2021,1,32.0,32.0,13,English Language Arts/Literacy,433.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both
6876,19,64212,6093454,2021,1,32.0,32.0,6,Mathematics,110.0,...,30.0,CALIFORNIA,19540.0,822.59,23.75,1620.0,107291.0,16966.0,1984.0,both


In [16]:
join.to_csv('../../data/processed/pov_and_all_school.csv')